# Keras Tuner

The Keras Tuner is a library that helps to pick the optimal set of hyperparameters for a TensorFlow program. The process of selecting the right set of hyperparameters for machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of an ML program. Hyperparameters are of two types:
1. **Model hyperparameters** influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

2022-11-30 16:56:21.962409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-30 16:56:21.962422: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255

## Define model

A hypermodel can be defined through two approaches:
- By using a model builder function
- By subclassing the `HyperModel` class of the keras tuner API

Two pre-defined HyperModel classes - `HyperXception` and `HyperResNet` can be also used for computer vision applications

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    #Tune the number of units in the first Dense layer
    #Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    
    #Tune learning rate (0.01, 0.001 or 0.0001)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                 loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    
    return model

## Instantiate the tuner and perform hypertuning

The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`.

In [5]:
tuner = kt.Hyperband(model_builder,
                    objective='val_accuracy',
                    max_epochs=10,
                    factor=3,
                    directory='my_dir',
                    project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


2022-11-30 16:56:23.367938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-30 16:56:23.367969: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-30 16:56:23.367983: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nsl54): /proc/driver/nvidia/version does not exist
2022-11-30 16:56:23.368112: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Hyperband determines the number of models to train in a bracket by computing 1 + logfactor(max_epochs) and rounding it up to the nearest integer.

In [6]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [7]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

#Get optimal Hyperparameter
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete. The optimal number of units in the first dendely-connected
layer is {best_hps.get('units')} and the ooptimal learning rate for the optimizer is {best_hps.get('learning_rate')}
""")

Trial 42 Complete [00h 00m 16s]
val_accuracy: 0.8785833120346069

Best val_accuracy So Far: 0.8910833597183228
Total elapsed time: 00h 01m 56s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first dendely-connected
layer is 384 and the ooptimal learning rate for the optimizer is 0.001



## train

In [8]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4950 - accuracy: 0.8240 - val_loss: 0.4058 - val_accuracy: 0.8572
Epoch 2/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3753 - accuracy: 0.8637 - val_loss: 0.3713 - val_accuracy: 0.8652
Epoch 3/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3339 - accuracy: 0.8783 - val_loss: 0.3452 - val_accuracy: 0.8753
Epoch 4/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3075 - accuracy: 0.8864 - val_loss: 0.3558 - val_accuracy: 0.8688
Epoch 5/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2902 - accuracy: 0.8922 - val_loss: 0.3198 - val_accuracy: 0.8853
Epoch 6/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2732 - accuracy: 0.8985 - val_loss: 0.3307 - val_accuracy: 0.8821
Epoch 7/50
1500/1500 [==============================] - 1s 972us/step - loss: 0.2585 - accuracy: 0.9028 - val_loss: 0.3158 - val_accurac

In [9]:
hypermodel = tuner.hypermodel.build(best_hps)

#Retrain
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/40
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4915 - accuracy: 0.8252 - val_loss: 0.4390 - val_accuracy: 0.8446
Epoch 2/40
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3695 - accuracy: 0.8656 - val_loss: 0.3656 - val_accuracy: 0.8648
Epoch 3/40
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3294 - accuracy: 0.8788 - val_loss: 0.4124 - val_accuracy: 0.8493
Epoch 4/40
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3046 - accuracy: 0.8874 - val_loss: 0.3472 - val_accuracy: 0.8753
Epoch 5/40
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2867 - accuracy: 0.8943 - val_loss: 0.3507 - val_accuracy: 0.8787
Epoch 6/40
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2707 - accuracy: 0.8990 - val_loss: 0.3157 - val_accuracy: 0.8875
Epoch 7/40
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2557 - accuracy: 0.9040 - val_loss: 0.3431 - val_accuracy:

In [10]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 0s 873us/step - loss: 0.5515 - accuracy: 0.8882
[test loss, test accuracy]: [0.5514851212501526, 0.8881999850273132]
